In [1]:
import pandas as pd
import numpy as np
from matplotlib import pyplot as plt
from deep_translator import GoogleTranslator
import numpy as np
from tqdm import tqdm
from num2words import num2words
batch = False

In [ ]:
responses_db = pd.read_csv('EDA/Simona/data/csv_files/buurtenqute_antwoord_dim.csv', encoding='latin1',skiprows=[1])
neighborhoods_db = pd.read_csv('EDA/Simona/data/csv_files/buurtenqute_buurt_dim.csv',encoding='latin1',skiprows=[1])
dates_db = pd.read_csv('EDA/Simona/data/csv_files/buurtenqute_datum_dim.csv',encoding='latin1')
report_n_responses_db = pd.read_csv('EDA/Simona/data/csv_files/buurtenqute_fact.csv',encoding='latin1')
composite_indicator_db = pd.read_csv('EDA/Simona/data/csv_files/buurtenqute_samengestelde_indicator_dim.csv',encoding='latin1',skiprows=[1])
questions_db = pd.read_csv('EDA/Simona/data/csv_files/buurtenqute_vraag_dim.csv',encoding='latin1',skiprows=[1])
green_index_df = pd.read_csv('EDA/Simona/data/csv_files/green_index.csv',encoding='latin1')
livability_index_df = pd.read_csv('EDA/Simona/data/csv_files/livability_index.csv',encoding='latin1')
print('successfully read csv files')

In [3]:
def dutch_to_english(text, batch=False):
    if len(text) > 1:
        try:
            if batch:
                keywords = []
                for words in text:
                    try:
                        keywords.append(GoogleTranslator(
                            source='auto', target='en').translate(words))
                    except:
                        num_word = num2words(float(words))
                        keywords.append(GoogleTranslator(
                            source='auto', target='en').translate(num_word))
                return keywords
            return GoogleTranslator(source='nl', target='en').translate(text)
        except:
            return text
    else:
        return np.nan

# Preprocessing
----

### 1) Responses survey

In [152]:
responses_db.head()

,AntwoordKey,Antwoordoptie,Antwoordomschrijving
0,1,1.0,Ja
1,2,2.0,Nee
2,3,3.0,Weet niet/geen mening
3,4,1.0,Ja
4,5,2.0,Nee


In [153]:
responses_db.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 8713 entries, 0 to 8712
Data columns (total 3 columns):
 #   Column                Non-Null Count  Dtype  
---  ------                --------------  -----  
 0   AntwoordKey           8713 non-null   int64  
 1   Antwoordoptie         8076 non-null   float64
 2   Antwoordomschrijving  8085 non-null   object 
dtypes: float64(1), int64(1), object(1)
memory usage: 204.3+ KB


In [154]:
# printing all missing values from the df
responses_db.isnull().sum()

AntwoordKey               0
Antwoordoptie           637
Antwoordomschrijving    628
dtype: int64

In [155]:
# droping missing values
# reset index and remove first column 'index'
responses_db.dropna(inplace=True)
responses_db.reset_index(inplace=True)
responses_db.drop('index',axis=1, inplace=True)

In [164]:
# translating columns from survey responses df
responses_db.columns = ['id_response','response_option','response_description']

In [ ]:
# translating values
responses_db['response_description'] = [dutch_to_english(x) for x in tqdm(responses_db['response_description'])]
print("***** completed translating data  *******")

In [167]:
responses_db

,id_response,response_option,response_description
0,1,1.0,Yes
1,2,2.0,No
2,3,3.0,Do not know / no opinion
3,4,1.0,Yes
4,5,2.0,No
...,...,...,...
8070,8709,8.0,Because of the house itself
8071,8710,8.0,Because of the house itself
8072,8711,3.0,Marriage / cohabitation
8073,8712,4.0,Family expansion


In [176]:
# saving translated df
responses_db.to_csv('C:/Users/dsimo/OneDrive/2022-23d-1fcmgt-reg-ai-01-SD222667/src/GreenScape/GreenScape/src/greenscape/data/responses_db.csv')

#### preprocessing survey responses done!
---

### 2) Neighborhoods survey

In [179]:
neighborhoods_db.head()

,BuurtKey,WijkOmschrijving,BuurtOmschrijving
0,1,Breda centrum,Valkenberg
1,2,Breda centrum,Chassé
2,3,Breda centrum,Fellenoord
3,4,Breda centrum,Schorsmolen
4,5,Breda centrum,Station


In [181]:
neighborhoods_db.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 61 entries, 0 to 60
Data columns (total 3 columns):
 #   Column             Non-Null Count  Dtype 
---  ------             --------------  ----- 
 0   BuurtKey           61 non-null     int64 
 1   WijkOmschrijving   61 non-null     object
 2   BuurtOmschrijving  61 non-null     object
dtypes: int64(1), object(2)
memory usage: 1.6+ KB


In [184]:
# resetting index and removing missing values
neighborhoods_db.dropna(inplace=True)
neighborhoods_db.reset_index(inplace=True)
neighborhoods_db.drop('index',axis=1, inplace=True)

In [190]:
# tranlsating columns
neighborhoods_db.columns = ['id_neighborhood','region','neighborhoods']

In [192]:
# saving df to .csv
neighborhoods_db.to_csv('C:/Users/dsimo/OneDrive/2022-23d-1fcmgt-reg-ai-01-SD222667/src/GreenScape/GreenScape/src/greenscape/data/neighborhoods_db.csv')

#### preprocessing neighborhoods survey done!
---

### 3) Dates survey  

In [193]:
dates_db.head()

,DatumKey,DatumDate,Datum,Italian_date,Jaar
0,20150101,2015-01-01 00:00:00,2015-01-01 00:00:00.000,2015-01-01 00:00:00,2015
1,20170101,2017-01-01 00:00:00,2017-01-01 00:00:00.000,2017-01-01 00:00:00,2017
2,20190101,2019-01-01 00:00:00,2019-01-01 00:00:00.000,2019-01-01 00:00:00,2019
3,20210101,2021-01-01 00:00:00,2021-01-01 00:00:00.000,2021-01-01 00:00:00,2021


In [201]:
dates_db.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 4 entries, 0 to 3
Data columns (total 5 columns):
 #   Column        Non-Null Count  Dtype 
---  ------        --------------  ----- 
 0   DatumKey      4 non-null      int64 
 1   DatumDate     4 non-null      object
 2   Datum         4 non-null      object
 3   Italian_date  4 non-null      object
 4   Jaar          4 non-null      int64 
dtypes: int64(2), object(3)
memory usage: 288.0+ bytes


In [56]:
# printing all missing values from the df
dates_db.isnull().sum().sum()

0

In [202]:
# droping duplicated columns and translating
dates_db.drop(columns = ['DatumDate','Italian_date'], inplace = True)
dates_db.columns = ['id_date','date','year']

In [204]:
# saving df to .csv 
dates_db.to_csv('C:/Users/dsimo/OneDrive/2022-23d-1fcmgt-reg-ai-01-SD222667/src/GreenScape/GreenScape/src/greenscape/data/dates_db.csv')

##### preprocessing dates done!
----

### 4) Report neighborhoods-responses

In [205]:
report_n_responses_db.head()

,DatumKey,BuurtKey,AntwoordKey,VraagKey,SamengesteldeIndicatorKey,BerekendeWaarde
0,20150101,1,-1,-1,-1,1.578410e-01
1,20150101,1,-1,-1,29,2.702958e+06
2,20150101,1,-1,-1,33,6.717265e+06
3,20150101,1,-1,-1,37,2.410750e-01
4,20150101,1,-1,-1,41,8.035460e-01


In [206]:
report_n_responses_db.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 373410 entries, 0 to 373409
Data columns (total 6 columns):
 #   Column                     Non-Null Count   Dtype  
---  ------                     --------------   -----  
 0   DatumKey                   373410 non-null  int64  
 1   BuurtKey                   373410 non-null  int64  
 2   AntwoordKey                373410 non-null  int64  
 3   VraagKey                   373410 non-null  int64  
 4   SamengesteldeIndicatorKey  373410 non-null  int64  
 5   BerekendeWaarde            373410 non-null  float64
dtypes: float64(1), int64(5)
memory usage: 17.1 MB


In [208]:
# printing all missing values from the df
report_n_responses_db.isnull().sum().sum()

0

In [209]:
# translating columns
report_n_responses_db.columns = ['id_date','id_neighborhood','id_response','id_question','id_compositeIndicator', 'calculated_value']

In [212]:
# saving df to csv file
report_n_responses_db.to_csv('C:/Users/dsimo/OneDrive/2022-23d-1fcmgt-reg-ai-01-SD222667/src/GreenScape/GreenScape/src/greenscape/data/report_n_responses_db.csv')

##### preprocessing neighborhoods-responses report done!
----

### 5) Composite indicator survey

In [214]:
composite_indicator_db.head()

,SamengesteldeIndicatorKey,IndicatorOmschrijving
0,1,Discriminatie
1,2,Discriminatie
2,3,Discriminatie
3,4,Discriminatie
4,5,Discriminatie


In [215]:
composite_indicator_db.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 140 entries, 0 to 139
Data columns (total 2 columns):
 #   Column                     Non-Null Count  Dtype 
---  ------                     --------------  ----- 
 0   SamengesteldeIndicatorKey  140 non-null    int64 
 1   IndicatorOmschrijving      140 non-null    object
dtypes: int64(1), object(1)
memory usage: 2.3+ KB


In [217]:
# translating columns and values
composite_indicator_db.columns = ['id_composite_indicator','indicator_description']
composite_indicator_db['indicator_description'] = [dutch_to_english(x) for x in tqdm(composite_indicator_db['indicator_description'])]
print("***** completed translating data  *******")

100%|██████████| 140/140 [01:08<00:00,  2.04it/s]

***** completed translating data  *******


In [223]:
# saving df to csv file
composite_indicator_db.to_csv('C:/Users/dsimo/OneDrive/2022-23d-1fcmgt-reg-ai-01-SD222667/src/GreenScape/GreenScape/src/greenscape/data/composite_indicator_db.csv')

##### preporcessing composite indicator data done!
----

### 6) Questions survey

In [4]:
questions_db.head()

,VraagKey,VraagOmschrijving,DeelVraagOmschrijving,Verkorte_Vraagstelling
0,1,Waar of wanneer was dit?,Bij het vinden van een (huur)huis (herberekend...,Oneerlijk/ongelijk behandeld bij vinden van ee...
1,2,In hoeverre bent u het eens of oneens met onde...,De gemeente doet een beroep op de buurtbewoner...,De gemeente doet een beroep op de buurtbewoner...
2,3,"Wanneer u hulp of zorg nodig heeft, kunt u dan...",mensen in de buurt,Kan terugvallen op mensen in de buurt
3,4,"Wanneer u hulp of zorg nodig heeft, kunt u dan...",vrienden,Kan terugvallen op vrienden
4,5,Krijgt u ergens hulp bij?,Uw lichamelijke gezondheid,Krijg hulp bij lichamelijke gezondheid


In [5]:
questions_db.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 2288 entries, 0 to 2287
Data columns (total 4 columns):
 #   Column                  Non-Null Count  Dtype 
---  ------                  --------------  ----- 
 0   VraagKey                2288 non-null   int64 
 1   VraagOmschrijving       2288 non-null   object
 2   DeelVraagOmschrijving   1988 non-null   object
 3   Verkorte_Vraagstelling  2120 non-null   object
dtypes: int64(1), object(3)
memory usage: 71.6+ KB


In [6]:
# printing all missing values from the df
questions_db.isnull().sum()

VraagKey                    0
VraagOmschrijving           0
DeelVraagOmschrijving     300
Verkorte_Vraagstelling    168
dtype: int64

In [7]:
# drop missing from df and resetting index
questions_db.dropna(inplace=True)
questions_db.reset_index(inplace=True)
questions_db.drop('index',axis=1, inplace=True)

In [8]:
# translating columns
questions_db.columns = ['id_question','question_description','subquestion_description','abbreviated_question']

In [9]:
# translating values
questions_db['question_description'] = [dutch_to_english(x) for x in tqdm(questions_db['question_description'])]
print("***** completed translating data  *******")
questions_db['subquestion_description'] = [dutch_to_english(x) for x in tqdm(questions_db['subquestion_description'])]
print("***** completed translating data  *******")
questions_db['abbreviated_question'] = [dutch_to_english(x) for x in tqdm(questions_db['abbreviated_question'])]
print("***** completed translating data  *******")

100%|██████████| 1844/1844 [14:25<00:00,  2.13it/s]


***** completed translating data  *******


100%|██████████| 1844/1844 [14:45<00:00,  2.08it/s]


***** completed translating data  *******


100%|██████████| 1844/1844 [14:21<00:00,  2.14it/s]

***** completed translating data  *******


In [11]:
questions_db.to_csv('C:/Users/dsimo/OneDrive/2022-23d-1fcmgt-reg-ai-01-SD222667/src/GreenScape/GreenScape/src/greenscape/data/questions_db.csv')

##### preprocessing questions data done!
----

### 7) Green index 

In [25]:
green_index_df.head()

,year,image_id,latitude,longitude,date,green_score,"cbs_grid_code,,"
0,2018,G5cqqBoLsR_OGx0kZyr_bQ,"51,581681454","4,7808467483",2018-09-01,8,E1129N3993
1,2018,G5cqqBoLsR_OGx0kZyr_bQ,"51,581681454","4,7808467483",2018-09-01,6,E1129N3993
2,2016,G5Durc0VsRb3goPZXWpmUQ,"51,620614209","4,7420451329",2016-08-01,86,E1103N4037
3,2016,G5Durc0VsRb3goPZXWpmUQ,"51,620614209","4,7420451329",2016-08-01,84,E1103N4037
4,2016,G5Durc0VsRb3goPZXWpmUQ,"51,620614209","4,7420451329",2016-08-01,49,E1103N4037


In [19]:
green_index_df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 298038 entries, 0 to 298037
Data columns (total 7 columns):
 #   Column           Non-Null Count   Dtype 
---  ------           --------------   ----- 
 0   year             298038 non-null  int64 
 1   image_id         298038 non-null  object
 2   latitude         298038 non-null  object
 3   longitude        298038 non-null  object
 4   date             298038 non-null  object
 5   green_score      298038 non-null  int64 
 6   cbs_grid_code,,  297984 non-null  object
dtypes: int64(2), object(5)
memory usage: 15.9+ MB


In [20]:
# printing all missing values from the df
green_index_df.isnull().sum()

year                0
image_id            0
latitude            0
longitude           0
date                0
green_score         0
cbs_grid_code,,    54
dtype: int64

In [21]:
# drop missing values and resetting indexes
green_index_df.dropna(inplace=True)
green_index_df.reset_index(inplace=True)
green_index_df.drop('index',axis=1, inplace=True)

In [22]:
len(green_index_df)

297984

In [23]:
# saving green_index to csv file
green_index_df.to_csv('C:/Users/dsimo/OneDrive/2022-23d-1fcmgt-reg-ai-01-SD222667/src/GreenScape/GreenScape/src/greenscape/data/green_index_db.csv')

##### preprocessing green index done!
---

### 8) Livability index  

In [26]:
livability_index_df.head()

,cbs_grid_code,livability_score
0,E1068N3982,2.423907
1,E1071N4012,2.376780
2,E1073N4018,2.533115
3,E1074N4011,2.405825
4,E1074N4017,2.502466


In [27]:
livability_index_df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 2937 entries, 0 to 2936
Data columns (total 2 columns):
 #   Column            Non-Null Count  Dtype  
---  ------            --------------  -----  
 0   cbs_grid_code     2937 non-null   object 
 1   livability_score  2937 non-null   float64
dtypes: float64(1), object(1)
memory usage: 46.0+ KB


In [28]:
livability_index_df.isnull().sum()

cbs_grid_code       0
livability_score    0
dtype: int64

In [29]:
livability_index_df.to_csv('C:/Users/dsimo/OneDrive/2022-23d-1fcmgt-reg-ai-01-SD222667/src/GreenScape/GreenScape/src/greenscape/data/livability_index_db.csv')